# RAG 優化

## 載入套件

In [3]:
! pip install bitsandbytes
! pip install accelerate

! pip install langchain langchain-chroma langchain-openai
! pip install langchain_huggingface
! pip install langchain_community
! pip install langchain_cohere


! pip install transformers
! pip install sentence_transformers

! pip install einops transformers_stream_generator
! pip install pymupdf

In [4]:
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import LLMChain

import re
import chromadb
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from pprint import pprint

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd

c:\Users\tlyu0419\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 直接跟LLM QA

In [2]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v1_0"


# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1000,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

c:\Users\tlyu0419\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\tlyu0419\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tlyu0419\.cache\huggingface\hub\models--shenzhi-wang--Llama3-8B-Chinese-Chat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you ei

In [2]:
template = """問題: {question}

Let's think step by step.

答案: """
prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 將問題透過參數化的方式帶入
question = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
print(llm_chain.invoke({"question": question})["text"])

問題: 模型的可解釋性真的很重要嗎? 為什麼他很重要?

Let's think step by step.

答案: 是的，模型的可解釋性非常重要。主要有以下幾個原因：

1. 理解和預測能力：模型的主要目的之一是幫助我們了解數據背後的趨勢、關係或行為。如果一個模型無法提供足夠多的信息來支持我們對數據的理解，它就失去了它的價值。因此，具有良好可解釋性的模型有助於提高我們對數據的洞察力並使我們能夠更好地做出決策。
2. 信任和可信度：在很多情況下，模型被用於支持重大決定，如醫療診斷、金融投資等。在這些場景中，人們需要相信模型的結果。如果模型不透明且不可解釋，人們可能會懷疑其準確性和可靠性，從而降低對其結果的信心。
3. 合規遵循：一些行業（例如金融業）要求公司使用符合監管規定的模型。這些模型通常需要能解釋其算法以及如何得出結論。因此，可解釋性對於遵守法律和法規至關重要。
4. 避免偏見和歧視：某些模型可能無意間嵌入了人類的偏見和歧視。只有當模型可以被理解時，才能發現這種偏差，並加以糾正。
5. 改進模型性能：通過分析模型的內部結構，研究人員可以找到改善模型性能的方法。這包括消除錯誤、減少過擬合、增加通用性等。



## 讀取文件

In [5]:
loader = PyMuPDFLoader('https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf')
ml_book = loader.load()

for i, p in enumerate(ml_book):
  ml_book[i].page_content = re.sub('\n', '', ml_book[i].page_content)

In [6]:
ml_book[:-5]

[Document(page_content='', metadata={'source': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'page': 0, 'total_pages': 226, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref package', 'producer': 'XeTeX 0.99996', 'creationDate': "D:20200423195438+08'00'", 'modDate': '', 'trapped': ''}),
 Document(page_content='序言机器学习对于改进产品、过程和研究有着很⼤的潜⼒。但是计算机通常无法解释他们的预测，这是采⽤机器学习的障碍。这本书是关于使机器学习模型及其决策可解释的。在探索了可解释性的概念之后，你将学习简单的、可解释的模型，例如决策树、决策规则和线性回归。后⾯⼏章重点介绍了解释⿊盒模型的模型⽆关的⼀般⽅法，如特征重要性和累积局部效应，以及⽤Shapley 值和LIME 解释单个实例预测。所有的解释⽅法进⾏了深⼊说明和批判性讨论。它们如何在⿊盒下⼯作的？它们的优缺点是什么？如何解释它们的输出？本书将使你能够选择并正确应⽤最适合你的机器学习项⽬的解释⽅法。这本书的重点是表格式数据(也称为关系数据或结构化数据) 的机器学习模型，较少涉及到计算机视觉和⾃然语⾔处理任务。建议机器学习从业者、数据科学家、统计学家和任何对使机器学习模型可解释的⼈阅读本书。你可以在leanpub.com 上购买PDF 和电⼦书版本(epub，mobi)。你可以在lulu.com 上购买印

## Naive RAG

### 放入向量資料庫

In [ ]:
! rm -rf document_store

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=50)
chunk_text = text_splitter.split_documents(ml_book)
len(chunk_text)
chunk_text[:5]

[Document(page_content='序言机器学习对于改进产品、过程和研究有着很⼤的潜⼒。但是计算机通常无法解释他们的预测，这是采⽤机器学习的障碍。这本书是关于使机器学习模型及其决策可解释的。在探索了可解释性的概念之后，你将学习简单的、可解释的模型，例如决策树、决策规则和线性回归。后⾯⼏章重点介绍了解释⿊盒模型的模型⽆关的⼀般⽅法，如特', metadata={'source': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'page': 1, 'total_pages': 226, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref package', 'producer': 'XeTeX 0.99996', 'creationDate': "D:20200423195438+08'00'", 'modDate': '', 'trapped': ''}),
 Document(page_content='解释的模型，例如决策树、决策规则和线性回归。后⾯⼏章重点介绍了解释⿊盒模型的模型⽆关的⼀般⽅法，如特征重要性和累积局部效应，以及⽤Shapley', metadata={'source': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_clu

In [13]:
embedding_func = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={'device':'cuda'
            })

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# load it into Chroma
db = Chroma.from_documents(chunk_text,
                embedding_func,
                persist_directory='document_store',
                collection_name='expai')

### 召回

In [8]:
# query it
query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
docs = db.similarity_search_with_score(query, k=5)
docs

[(Document(metadata={'author': '', 'creationDate': "D:20200423195438+08'00'", 'creator': 'LaTeX with hyperref package', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 18, 'producer': 'XeTeX 0.99996', 'source': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'subject': '', 'title': '', 'total_pages': 226, 'trapped': ''}, page_content='和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道'),
  61.600833892822266),
 (Document(metadata={'author': '', 'creationDate': "D:20200423195438+08'00'", 'creator': 'LaTeX with hyperref package', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'format': 'PDF 1.5', 'keyword

### 生成

In [9]:
# Prompt 模板
template = """
<s>你是資深的資料科學家，請參考下方提供的資訊回應使用者提出的問題
資訊: {context}

[INST] {question}
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})
query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
result = qa_chain({"query": query})
print(result["result"])

<s>你是資深的資料科學家，請參考下方提供的資訊回應使用者提出的問題
資訊: 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道

和Explainable 是有意义的。我们将使⽤Explainable 来描述对单个实例预测的解释。2.1可解释性的重要性如果⼀个机器学习模型运⾏良好，为什么我们仅仅信任该模型⽽忽略为什么做出特定的决策呢？诸如分类准确性(Classification Accuracy)

个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。

的整体认知来理解模型是如何做出决策的。哪些特征很重要，以及它们之间有什么样的交互作⽤？全局的模型可解释性有助于基于特征理解⽬标结果的分布。但在实践中很难实现全局模型可解释性，任何超过⼏个参数或权重的模型都不可能适合⼈的短期记忆。就⽐如说，我认为你很难想象⼀个具有5

不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2全局、整体的模型可解释性训练好的模型如何进⾏预测？⼀旦能理解整个模型，就可以将模型描述为可解释的(Lipton，2016[7])。要解释全局模型输出，你需要训练好的模型、算法知识和数据。这种级别的可解释性是

[INST] 模型的可解釋性真的很重要嗎? 為什麼他很重要?

模型的可解釋性很重要，因為它有助於我們了解模型如何做出決策，以及哪些特徵非常重要，並且它們之間存在何種相互作用。這有助於基於特徵理解目標結果的分佈。然而，在實踐中，很難實現整體模型可解釋性，任何超過幾個參數或權重的模型都無法適合人類的短期記憶。就像說，我認為你想象一個具有 5,000 個特徵是很困難的。


In [11]:
# 參考的相關資訊
pprint(result["source_documents"])

[Document(metadata={'author': '', 'creationDate': "D:20200423195438+08'00'", 'creator': 'LaTeX with hyperref package', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 18, 'producer': 'XeTeX 0.99996', 'source': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'subject': '', 'title': '', 'total_pages': 226, 'trapped': ''}, page_content='和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道'),
 Document(metadata={'author': '', 'creationDate': "D:20200423195438+08'00'", 'creator': 'LaTeX with hyperref package', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'p

## 優化

### Chunk

In [ ]:
! rm -rf document_store

In [ ]:
del db

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunk_text = text_splitter.split_documents(ml_book)
len(chunk_text)
chunk_text[:5]

[Document(page_content='序言机器学习对于改进产品、过程和研究有着很⼤的潜⼒。但是计算机通常无法解释他们的预测，这是采⽤机器学习的障碍。这本书是关于使机器学习模型及其决策可解释的。在探索了可解释性的概念之后，你将学习简单的、可解释的模型，例如决策树、决策规则和线性回归。后⾯⼏章重点介绍了解释⿊盒模型的模型⽆关的⼀般⽅法，如特征重要性和累积局部效应，以及⽤Shapley 值和LIME 解释单个实例预测。所有的解释⽅法进⾏了深⼊说明和批判性讨论。它们如何在⿊盒下⼯作的？它们的优缺点是什么？如何解释它们的输出？本书将使你能够选择并正确应⽤最适合你的机器学习项⽬的解释⽅法。这本书的重点是表格式数据(也称为关系数据或结构化数据) 的机器学习模型，较少涉及到计算机视觉和⾃然语⾔处理任务。建议机器学习从业者、数据科学家、统计学家和任何对使机器学习模型可解释的⼈阅读本书。你可以在leanpub.com 上购买PDF 和电⼦书版本(epub，mobi)。你可以在lulu.com 上购买印刷版。关于我：我叫克⾥斯托夫·莫纳(Christoph', metadata={'source': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'file_path': 'https://raw.githubusercontent.com/cathayins/llama_club/1341a938c0857d71c900b7b580e6d63aeb28e2da/語料庫/可解释的机器学习.pdf', 'page': 1, 'total_pages': 226, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref package', 'producer': 'XeTeX 0.99996', 'creationDate': "D:20200423195438+08'00'", 'modDate': '', 'trapped': ''}

In [ ]:
db = Chroma.from_documents(chunk_text,embedding_func)

In [ ]:
# Prompt 模板
template = """
<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊: {context}

[INST] {question}
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})
query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
result = qa_chain({"query": query})
print(result["result"])

<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊: Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez

个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。

不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2全局、整体的模型可解释性训练好的模型如何进⾏预测？⼀旦能理解整个模型，就可以将模型描述为可解释的(Lipton，2016[7])。要解释全局模型输出，你需要训练好的模型、算法知识和数据。这种级别的可解释性是基于对模型特征和每个学习部分(如权重、其他参数和结构) 的整体认知来理解模型是如何做出决策的。哪些特征很重要，以及它们之间有什么样的交互作⽤？全局的模型可解释性有助于基于特征理解⽬标结果的分布。但在实践中很难实现全局模型可解释性，任何超过⼏个参数或权重的模型都不可能适合⼈的短期记忆。就⽐如说，我认为你很难想象⼀个具有5 个特征的线性模型，因为这意味着要想象在5 维空间中绘制估

### Prompt

In [ ]:
# Prompt 模板
template = """
<s> 你是熱心的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問。多回答他一些內容，幫助他更好的學習資料科學的知識與技術，做得好的話他會給你 1000 元小費
資訊: {context}
[INST] {question}
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [14]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})
query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
result = qa_chain({"query": query})
print(result["result"])

<s> 你是熱心的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問。多回答他一些內容，幫助他更好的學習資料科學的知識與技術，做得好的話他會給你 1000 元小費
資訊: Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez

个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。

不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2全局、整体的模型可解释性训练好的模型如何进⾏预测？⼀旦能理解整个模型，就可以将模型描述为可解释的(Lipton，2016[7])。要解释全局模型输出，你需要训练好的模型、算法知识和数据。这种级别的可解释性是基于对模型特征和每个学习部分(如权重、其他参数和结构) 的整体认知来理解模型是如何做出决策的。哪些特征很重要，以及它们之间有什么样的交互作⽤？全局的模型可解释性有助于基于特征理解⽬标结果的分布。但在实践中很难实现全局模型可解释性，任何超过⼏个参数或权重的模型都不可能适合⼈的短期记忆

### 召回數量

In [ ]:
# Prompt 模板
template = """
<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊: {context}

[INST] {question}
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [15]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 10}), # <<<----- 改了這邊的參數
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
result = qa_chain({"query": query})
print(result["result"])

<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊: Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez

个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。

不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2全局、整体的模型可解释性训练好的模型如何进⾏预测？⼀旦能理解整个模型，就可以将模型描述为可解释的(Lipton，2016[7])。要解释全局模型输出，你需要训练好的模型、算法知识和数据。这种级别的可解释性是基于对模型特征和每个学习部分(如权重、其他参数和结构) 的整体认知来理解模型是如何做出决策的。哪些特征很重要，以及它们之间有什么样的交互作⽤？全局的模型可解释性有助于基于特征理解⽬标结果的分布。但在实践中很难实现全局模型可解释性，任何超过⼏个参数或权重的模型都不可能适合⼈的短期记忆。就⽐如说，我认为你很难想象⼀个具有5 个特征的线性模型，因为这意味着要想象在5 维空间中绘制估

### 召回方式

In [ ]:
# Prompt 模板
template = """
<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊: {context}

[INST] {question}
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 5}, search_type="mmr"),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
result = qa_chain({"query": query})
print(result["result"])

<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊: Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez

个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。

第⼆章可解释性15以下场景说明了我们何时不需要甚⾄不希望机器学习模型的可解释性。• 如果模型没有重大影响，则不需要解释性。想象⼀下，⼀个名为Mike 的⼈正在做⼀个机器学习⽅⾯的项⽬，根据Facebook 的数据预测他的朋友们下⼀个假期会去哪⾥。Mike 就是喜欢有依据地推测朋友们会去哪⾥度假，从⽽让他的朋友们吃惊。如果模型是错误的也没有问题(最坏的情况是，Mike 有点尴尬)；如果Mike 不能解释模型的输出，那也没有问题。在这种情况下，没有可解释性是完全可以的。如果Mike 开始围绕这些度假⽬的地的预测建⽴业务，情况将会改变。如果模型是错误的，企业可能会赔钱，或者模型可能会因为种族偏见⽽对某些⼈变得更糟。⼀旦模型产⽣重⼤影响，⽆论是⾦融还是社会，可解释性就变得很重

### 排序

In [18]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

pairs = [['what is panda?', 'hi'], 
  ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

c:\Users\tlyu0419\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\tlyu0419\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tlyu0419\.cache\huggingface\hub\models--BAAI--bge-reranker-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to

tensor([-5.6085,  5.7623])


In [20]:
# query it
query = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
docs = db.similarity_search_with_score(query, k=10)
contents = [d[0].page_content for d in docs]
contents

['Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez',
 '个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。',
 '不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2全局、整体的模型可解释性训练好的模型如何进⾏预测？⼀旦能理解整个模型，就可以将模型描述为可解释的(Lipton，2016[7])。要解释全局模型输出，你需要训练好的模型、算法知识和数据。这种级别的可解释性是基于对模型特征和每个学习部分(如权重、其他参数和结构) 的整体认知来理解模型是如何做出决策的。哪些特征很重要，以及它们之间有什么样的交互作⽤？全局的模型可解释性有助于基于特征理解⽬标结果的分布。但在实践中很难实现全局模型可解释性，任何超过⼏个参数或权重的模型都不可能适合⼈的短期记忆。就⽐如说，我认为你很难想象⼀个具有5 个特征的线性模型，因为这意味着要想象在5 维空间中绘制估计的超平⾯。甚⾄任何超过3',
 '第⼆章可解释性15以下场景说明了我们何

In [21]:
pairs = [[query, c] for c in contents]
pairs

[['模型的可解釋性真的很重要嗎? 為什麼他很重要?',
  'Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez'],
 ['模型的可解釋性真的很重要嗎? 為什麼他很重要?',
  '个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。'],
 ['模型的可解釋性真的很重要嗎? 為什麼他很重要?',
  '不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2全局、整体的模型可解释性训练好的模型如何进⾏预测？⼀旦能理解整个模型，就可以将模型描述为可解释的(Lipton，2016[7])。要解释全局模型输出，你需要训练好的模型、算法知识和数据。这种级别的可解释性是基于对模型特征和每个学习部分(如权重、其他参数和结构) 的整体认知来理解模型是如何做出决策的。哪些特征很重要，以及它们之间有什么样的交互作⽤？全局的模型可解释性有助于基于特征理解⽬标结果的分布。但在实践中很难实现全局模型可解释性，任何超过⼏个参数或权重的模型都不可能适

In [23]:
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([ 5.0519,  3.3723,  2.0901,  4.7712,  2.9887,  1.1566, -0.6153, -0.5049,
         2.0030,  1.9816])


In [26]:
pairs = [[query, c, s] for c, s in zip(contents, scores)]
pd.DataFrame(pairs)

,0,1,2
0,模型的可解釋性真的很重要嗎? 為什麼他很重要?,Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和...,5.051896
1,模型的可解釋性真的很重要嗎? 為什麼他很重要?,个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例...,3.372279
2,模型的可解釋性真的很重要嗎? 為什麼他很重要?,不太容易理解，对其内部⼯作机制的探究是正在进⾏的研究重点，它们被认为是低透明度的。2.3.2...,2.090078
3,模型的可解釋性真的很重要嗎? 為什麼他很重要?,第⼆章可解释性15以下场景说明了我们何时不需要甚⾄不希望机器学习模型的可解释性。• 如果模型...,4.771186
4,模型的可解釋性真的很重要嗎? 為什麼他很重要?,第二章可解释性对可解释性是没有数学上定义的。我⽐较喜欢Miller (2017)[3] 的(...,2.988652
5,模型的可解釋性真的很重要嗎? 為什麼他很重要?,第⼆章可解释性21• 确定性(Certainty)：解释是否反映了机器学习模型的确定性？许多...,1.156630
6,模型的可解釋性真的很重要嗎? 為什麼他很重要?,第⼆章可解释性18块层⾯上理解某些模型。并⾮所有模型都可以在参数级别上解释。对于线性模型，可...,-0.615347
7,模型的可解釋性真的很重要嗎? 為什麼他很重要?,本质上可解释模型：解释⿊盒模型的⼀个解决⽅案是⽤可解释模型(全局地或局部地) 对其进⾏近似。...,-0.504875
8,模型的可解釋性真的很重要嗎? 為什麼他很重要?,和Guestrin，2016)：• 模型的灵活性：解释⽅法可以与任何机器学习模型⼀起使⽤，例...,2.002981
9,模型的可解釋性真的很重要嗎? 為什麼他很重要?,上发表的有关可解释性的新论⽂。本书以⼀些(反乌托邦式的) 短篇⼩说作为开篇，这些短篇⼩说不是...,1.981618


In [27]:
template = """
<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊:
1. Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez'],
2. 第⼆章可解释性15以下场景说明了我们何时不需要甚⾄不希望机器学习模型的可解释性。• 如果模型没有重大影响，则不需要解释性。想象⼀下，⼀个名为Mike 的⼈正在做⼀个机器学习⽅⾯的项⽬，根据Facebook 的数据预测他的朋友们下⼀个假期会去哪⾥。Mike 就是喜欢有依据地推测朋友们会去哪⾥度假，从⽽让他的朋友们吃惊。如果模型是错误的也没有问题(最坏的情况是，Mike 有点尴尬)；如果Mike 不能解释模型的输出，那也没有问题。在这种情况下，没有可解释性是完全可以的。如果Mike 开始围绕这些度假⽬的地的预测建⽴业务，情况将会改变。如果模型是错误的，企业可能会赔钱，或者模型可能会因为种族偏见⽽对某些⼈变得更糟。⼀旦模型产⽣重⼤影响，⽆论是⾦融还是社会，可解释性就变得很重要了。•
3. 个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最近邻可以为你提供很好的解释。
4. 第二章可解释性对可解释性是没有数学上定义的。我⽐较喜欢Miller (2017)[3] 的(⾮数学的) 定义：可解释性是⼈们能够理解决策原因的程度。另⼀种定义是[4]：可解释性是指⼈们能够⼀致地预测模型结果的程度。机器学习模型的可解释性越⾼，⼈们就越容易理解为什么做出某些决策或预测。如果⼀个模型的决策⽐另⼀个模型的决策能让⼈更容易理解，那么它就⽐另⼀个模型有更⾼的解释性。我们将在后⽂中同时使⽤Interpretable 和Explainable 这两个术语来描述可解释性。像Miller (2017) ⼀样，区分术语Interpretable 和Explainable 是有意义的。我们将使⽤Explainable 来描述对单个实例预测的解释。2.1可解释性的重要性如果⼀个机器学习模型运⾏良好，为什么我们仅仅信任该模型⽽忽略为什么做出特定的决策呢？诸如分类准确性(Classification Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez
[INST] {question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# 使用 LLM Chain 將 Prompt 與 LLM 串接起來
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 將問題透過參數化的方式帶入
question = "模型的可解釋性真的很重要嗎? 為什麼他很重要?"
print(llm_chain.invoke({"question": question})["text"])

<s>你是資深的資料科學家，請參考下方提供的資訊，並使用繁體中文回應使用者的提問
資訊:
1. Accuracy) 之类的单⼀指标⽆法完整地描述⼤多数实际任务。(Doshi-Velez 和Kim，2017[5])我们深⼊探讨可解释性的重要性。当涉及到预测模型时，你需要作出权衡：你是只想知道预测是什么？例如，客户流失的概率或某种药物对病⼈的疗效。还是想知道为什么做出这样的预测？这种情况下可能为了可解释性付出预测性能下降的代价。在某些情况下，你不必关⼼为什么要做出这样的预测，只要知道模型在测试数据集的预测性能良好就⾜够了。但是在其他情况下，了解“为什么” 可以帮助你更多地了解问题、数据以及模型可能失败的原因。有些模型可能不需要解释，因为它们是在低风险的环境中使⽤的，这意味着错误不会造成严重后果(例如，电影推荐系统)，或者该⽅法已经被⼴泛研究和评估(例如，光学字符识别OCR)。对可解释性的需求来⾃问题形式化的不完整性[5]，这意味着对于某些问题或任务，仅仅获得预测结果是不够的。该模型还必须解释是怎么获得这个预测的，因为正确的预测只部分地解决了你的原始问题。以下原因推动了对可解释性(Interpretability) 和解释(Explanations) 的需求(Doshi-Velez'],
2. 第⼆章可解释性15以下场景说明了我们何时不需要甚⾄不希望机器学习模型的可解释性。• 如果模型没有重大影响，则不需要解释性。想象⼀下，⼀个名为Mike 的⼈正在做⼀个机器学习⽅⾯的项⽬，根据Facebook 的数据预测他的朋友们下⼀个假期会去哪⾥。Mike 就是喜欢有依据地推测朋友们会去哪⾥度假，从⽽让他的朋友们吃惊。如果模型是错误的也没有问题(最坏的情况是，Mike 有点尴尬)；如果Mike 不能解释模型的输出，那也没有问题。在这种情况下，没有可解释性是完全可以的。如果Mike 开始围绕这些度假⽬的地的预测建⽴业务，情况将会改变。如果模型是错误的，企业可能会赔钱，或者模型可能会因为种族偏见⽽对某些⼈变得更糟。⼀旦模型产⽣重⼤影响，⽆论是⾦融还是社会，可解释性就变得很重要了。•
3. 个点。模型是否可解释仅取决于是否可以“解释” 数据集中的单个实例的问题。我认为，如果⼀个实例包含成百上千个特征，那么它是不可解释的。但是，如果你有少量特征或有将实例简化为最重要特征的⽅法，那么展⽰k-最